# Application of Bootstrap samples in Random Forest

In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

 <li> Load the boston house dataset </li>

In [2]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

### Task: 1
<font color='red'><b>Step 1 Creating samples: </b></font> Randomly create 30 samples from the whole boston data points.
<ol>
<li>Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points</li>
<li>Ex: For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly consider we have selected [4, 5, 7, 8, 9, 3] now we will replciate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]</li>
<li> we create 30 samples like this </li>
<li> Note that as a part of the Bagging when you are taking the random samples make sure each of the sample will have                different set of columns</li>
<li> Ex: assume we have 10 columns for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample [7, 9, 1, 4, 5, 6, 2] and so on...</li>
<li> Make sure each sample will have atleast 3 feautres/columns/attributes</li>
</ol>

<font color='red'><b>Step 2 Building High Variance Models on each of the sample and finding train MSE value:</b></font> Build a DecisionTreeRegressor on each of the sample.
<ol><li>Build a regression trees on each of 30 samples.</li>
<li>computed the predicted values of each data point(506 data points) in your corpus.</li>
<li> predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

<font color='red'><b>Step 3 Calculating the OOB score :</b></font>
<ol>
<li>Computed the predicted values of each data point(506 data points) in your corpus.</li>
<li>Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{k}\sum_{\text{k= model which was buit on samples not included } x^{i}}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

### Task: 2
<pre>
<font color='red'><b>Computing CI of OOB Score and Train MSE</b></font>
<ol>
<li> Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score </li>
<li> After this we will have 35 Train MSE values and 35 OOB scores </li>
<li> using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score </li>
<li> you need to report CI of MSE and CI of OOB Score </li>
<li> Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel</li>
</ol>
</pre>
### Task: 3
<pre>
<font color='red'><b>Given a single query point predict the price of house.</b></font>

<li>Consider xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] Predict the house price for this point as mentioned in the step 2 of Task 1. </li>
</pre>

## Task: 1

In [3]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import random
from tqdm import tqdm

In [4]:
# Converting x values to dataframe
data = pd.DataFrame(data=x[:,:], index= range(len(x)), columns=boston.feature_names)

In [5]:
class Bootstrap_random_forest:
    def __init__(self,x,y,n = 30):
        self.n = n
        self.x = x
        self.y = y
        self.X_n_sample = {}
        self.Y_n_sample = {}
        self.column_sample_index = {}
        self.row_sample_index = {}
        self.y_pred = []
        self.y_pred_oob = []
        
    def create_n_samples(self):
        data_size_60 = (int)(0.6*self.x.shape[0])
        data_size_40 = self.x.shape[0] - data_size_60
    
        for i in range(self.n):
#       column sampling
            idx_col = random.sample(range(data.shape[1]),random.randrange(3, data.shape[1]))
            self.column_sample_index[i] = idx_col 
        
#       row sampling
            idx = random.sample(range(self.x.shape[0]),data_size_60)
            idx2 = random.sample(idx,data_size_40)
            idx_row = idx + idx2
            self.row_sample_index[i] = idx_row
        
            sample_x = self.x.iloc[idx_row,idx_col].values
            sample_y = self.y[idx_row]
        
            self.X_n_sample[i] = sample_x
            self.Y_n_sample[i] = sample_y
    
    def train_model(self):
        y_pred_total = np.zeros(506)
        regressor = DecisionTreeRegressor(random_state=0)
        for i in range(self.n):
            regressor.fit(self.X_n_sample[i],self.Y_n_sample[i])
            y_pred_sample = regressor.predict(self.x.iloc[:,self.column_sample_index[i]])
        #     print(y_pred_sample)
            y_pred_total = np.add(y_pred_sample,y_pred_total)
        self.y_pred = (1/30)*y_pred_total
    
    def train_model_oob(self):
        for i in range(self.x.shape[0]):
            y_pred_sample = 0
            k = 0
            regressor = DecisionTreeRegressor(random_state=0)
            for j in range(self.n):
                if i  not in self.row_sample_index[j]:
                    k+=1
                    regressor.fit(self.X_n_sample[j],self.Y_n_sample[j])
#                     print(self.column_sample_data[j].shape)
                    y_pred_sample += regressor.predict(self.x.iloc[:,self.column_sample_index[j]])
            #         print(y_pred_sample)
            self.y_pred_oob.append((1/k)*y_pred_sample)
            
    def predict_sample(self,data):
        y_pred_total = np.zeros(len(data))
        regressor = DecisionTreeRegressor(random_state=0)
        for i in range(self.n):
            regressor.fit(self.X_n_sample[i],self.Y_n_sample[i])
            y_pred_sample = regressor.predict(data[:,self.column_sample_index[i]])
        #     print(y_pred_sample)
            y_pred_total = np.add(y_pred_sample,y_pred_total)
        y_pred = (1/30)*y_pred_total
        return y_pred
    
    def mean_square_error(self,y_orig):
        return (1/506)*np.sum(np.subtract(y_orig,self.y_pred) )
    
    def mean_square_error_oob(self,y_orig):
        return (1/506)*np.sum(np.subtract(y_orig,self.y_pred_oob) )

In [6]:
model = Bootstrap_random_forest(data,y,30)

### Step 1: Creating samples: Randomly create 30 samples from the whole boston data points.¶

In [7]:
model.create_n_samples()

#### Step 2 Building High Variance Models on each of the sample and finding train MSE value`

In [8]:
model.train_model()

In [9]:
model.mean_square_error(y)

-0.08279268706968855

### Step 3 Calculating the OOB score :

In [10]:
model.train_model_oob()

In [11]:
model.mean_square_error_oob(y)

-41.21965559590377

## Task: 2

In [12]:
mse = []
oob_score = []
for i in tqdm(range(35)):
    model = Bootstrap_random_forest(data,y,30)
    model.create_n_samples()
    model.train_model()
    mse.append(model.mean_square_error(y))
    model.train_model_oob()
    oob_score.append(model.mean_square_error_oob(y))

100%|██████████| 35/35 [16:02<00:00, 27.60s/it]


In [13]:
print('MSE Mean: ',np.array(mse).mean(),'MSE Std: ',np.array(mse).std())
print('OOB Score Mean: ',np.array(oob_score).mean(),'OOB Score Std: ',np.array(oob_score).std())

MSE Mean:  0.0005637626761705887 MSE Std:  0.03298793171491998
OOB Score Mean:  0.2963369381016723 OOB Score Std:  16.728015839311727


In [14]:
print('Confidence Interval of MSE: [',np.array(mse).mean()-2*np.array(mse).std(),',',np.array(mse).mean()+2*np.array(mse).std(),']')
print('Confidence Interval of OOB Score: [',np.array(oob_score).mean()-2*np.array(oob_score).std(),',',np.array(oob_score).mean()+2*np.array(oob_score).std(),']')

Confidence Interval of MSE: [ -0.06541210075366938 , 0.06653962610601055 ]
Confidence Interval of OOB Score: [ -33.15969474052178 , 33.75236861672513 ]


## Task: 3

In [15]:
xq = np.array([[0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60]])
model.predict_sample(xq)

array([20.12333333])